In [3]:
#@title installs
!pip install -q optuna
!pip install -q google-colab


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement google-colab (from versions: none)

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for google-colab


In [2]:
#@title imports

import h5py
import re
import time

from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from typing import Dict

import optuna
import torch
import torch.nn as nn
import wandb

from torch.utils.data import DataLoader, TensorDataset

ModuleNotFoundError: No module named 'google.colab'

In [3]:
#@title utils

# ===================
# function: load data
# ===================
def load_data(hdf5_path: str, verbose: bool=True) -> Dict:
    """
    load data from hdf5 file
    """
    start_time = time.time()
    print("loading data from HDF5 file...")

    with h5py.File(hdf5_path, "r") as f:
        # load the sentences
        sentences = [s.decode('utf-8') for s in f["sentences"][:]]

        # load embeddings for each layer
        layer_8 = torch.tensor(f["layer_8"][:], dtype=torch.float32)
        layer_16 = torch.tensor(f["layer_16"][:], dtype=torch.float32)
        layer_24 = torch.tensor(f["layer_24"][:], dtype=torch.float32)

    if verbose:
        print(f"Loaded {len(sentences)} sentences and their embeddings")
        print(f"Layer 8 embeddings shape: {layer_8.shape}")
        print(f"Layer 16 embeddings shape: {layer_16.shape}")
        print(f"Layer 24 embeddings shape: {layer_24.shape}")
        print(f"Data loading completed in {time.time() - start_time:.2f} seconds")

    return {"sentences": sentences, "layer_8": layer_8, "layer_16": layer_16, "layer_24": layer_24}

# =========================
# function: parse sentences
# =========================
def parse_sentences(sentences: list, verbose: bool=True) -> None:
    """
    parse sentences
    """
    start_time = time.time()
    print("parsing sentences into triplets...")

    # primary pattern for "The X is Y the Z." format
    pattern = r"The (.*?) is (.*?) the (.*?)\."

    triplets = []
    valid_indices = []

    for i, sentence in enumerate(sentences):
        match = re.search(pattern, sentence)
        if match:
            obj1, relation, obj2 = match.groups()
            triplets.append((obj1.strip(), relation.strip(), obj2.strip()))
            valid_indices.append(i)
        else:
            # secondary pattern for relations without "the" (like "on", "facing")
            # look for patterns like "The table is on the chair." or "The table is facing the lamp."
            alt_pattern = r"The (.*?) is (.*) (chair|lamp|table)\."
            match = re.search(alt_pattern, sentence)
            if match:
                obj1, relation, obj2 = match.groups()
                triplets.append((obj1.strip(), relation.strip(), obj2.strip()))
                valid_indices.append(i)
            else:
                # last attempt with very general pattern
                try:
                    # split the sentence
                    # "The table is connected to the lamp." → ["The", "table", "is", "connected", "to", "the", "lamp."]
                    words = sentence.split()
                    if len(words) >= 5 and words[0].lower() == "the" and words[2].lower() == "is":
                        obj1 = words[1]

                        # find the last occurrence of "the" to locate obj2
                        if "the" in words[3:]:
                            last_the_idx = len(words) - 1 - words[::-1].index("the")
                            obj2 = words[last_the_idx+1].rstrip('.')
                            relation = " ".join(words[3:last_the_idx])
                            triplets.append((obj1.strip(), relation.strip(), obj2.strip()))
                            valid_indices.append(i)
                        else:
                            # if no "the" found, assume the last word is the object
                            obj2 = words[-1].rstrip('.')
                            relation = " ".join(words[3:-1])
                            triplets.append((obj1.strip(), relation.strip(), obj2.strip()))
                            valid_indices.append(i)
                    else:
                        print(f"Failed to parse: {sentence}")
                except Exception as e:
                    print(f"Error parsing: {sentence}, Error: {e}")

    if verbose:
        print(f"Successfully parsed {len(triplets)} triplets")
        print(f"Parsing completed in {time.time() - start_time:.2f} seconds")

        # Display a few parsed triplets for verification
        print("\nSample triplets:")
        for i in range(min(5, len(triplets))):
            print(f"{i}: {triplets[i]}")

    return {"triplets": triplets, "valid_indices": valid_indices}


# =========================
# function: encode triplets
# =========================
def encode_triplets(triplets, verbose=True):
    """
    encode triplets for model training
    """
    start_time = time.time()
    print("Encoding triplets...")

    # extract separate components
    objects1, relations, objects2 = zip(*triplets)

    # filter out empty strings
    objects1 = [obj if obj else "UNKNOWN" for obj in objects1]
    relations = [rel if rel else "UNKNOWN" for rel in relations]
    objects2 = [obj if obj else "UNKNOWN" for obj in objects2]

    # encode each component
    obj1_encoder = LabelEncoder()
    rel_encoder = LabelEncoder()
    obj2_encoder = LabelEncoder()

    obj1_labels = obj1_encoder.fit_transform(objects1)
    rel_labels = rel_encoder.fit_transform(relations)
    obj2_labels = obj2_encoder.fit_transform(objects2)

    labels = {"object_1": obj1_labels, "relation": rel_labels, "object_2": obj2_labels}

    # create mapping dictionaries for later analysis
    obj1_mapping = dict(zip(obj1_encoder.classes_, range(len(obj1_encoder.classes_))))
    rel_mapping = dict(zip(rel_encoder.classes_, range(len(rel_encoder.classes_))))
    obj2_mapping = dict(zip(obj2_encoder.classes_, range(len(obj2_encoder.classes_))))

    mappings = {"object_1": obj1_mapping, "relation": rel_mapping, "object_2": obj2_mapping}

    if verbose:
        print(f"Found {len(obj1_mapping)} unique objects as subject")
        print(f"Found {len(rel_mapping)} unique relations")
        print(f"Found {len(obj2_mapping)} unique objects as object")

        # Print some of the unique relations
        print("\nSample relations:")
        sample_relations = list(rel_mapping.keys())[:10]
        for i, rel in enumerate(sample_relations):
            print(f"{i}: {rel}")

        print(f"Encoding completed in {time.time() - start_time:.2f} seconds")

    return labels, mappings


# ============================
# function: prepare data split
# ============================
def prepare_data_split(layer_data, valid_indices, labels, test_size=0.2):
    """
    prepare single train/test split for all labels
    """
    # filter embeddings to keep only valid indices
    X = layer_data[valid_indices]

    # create a single train/test split for all labels
    X_train, X_test, y_obj1_train, y_obj1_test, y_rel_train, y_rel_test, y_obj2_train, y_obj2_test = train_test_split(
        X,
        labels["object_1"],
        labels["relation"],
        labels["object_2"],
        test_size=test_size,
        random_state=42
        )

    # convert to PyTorch tensors
    X_train_tensor = X_train.clone().detach().float()
    X_test_tensor = X_test.clone().detach().float()

    y_obj1_train_tensor = torch.tensor(y_obj1_train, dtype=torch.long)
    y_obj1_test_tensor = torch.tensor(y_obj1_test, dtype=torch.long)

    y_rel_train_tensor = torch.tensor(y_rel_train, dtype=torch.long)
    y_rel_test_tensor = torch.tensor(y_rel_test, dtype=torch.long)

    y_obj2_train_tensor = torch.tensor(y_obj2_train, dtype=torch.long)
    y_obj2_test_tensor = torch.tensor(y_obj2_test, dtype=torch.long)

    return {
        'X_train': X_train_tensor,
        'X_test': X_test_tensor,
        'y_obj1_train': y_obj1_train_tensor,
        'y_obj1_test': y_obj1_test_tensor,
        'y_rel_train': y_rel_train_tensor,
        'y_rel_test': y_rel_test_tensor,
        'y_obj2_train': y_obj2_train_tensor,
        'y_obj2_test': y_obj2_test_tensor
        }

# =============================
# function: create data loaders
# =============================
def create_data_loaders(split_data, batch_size=32):
    """
    create data loaders for training and testing
    """
    # create training datasets
    train_obj1_dataset = TensorDataset(split_data['X_train'], split_data['y_obj1_train'])
    train_rel_dataset = TensorDataset(split_data['X_train'], split_data['y_rel_train'])
    train_obj2_dataset = TensorDataset(split_data['X_train'], split_data['y_obj2_train'])

    # create test datasets
    test_obj1_dataset = TensorDataset(split_data['X_test'], split_data['y_obj1_test'])
    test_rel_dataset = TensorDataset(split_data['X_test'], split_data['y_rel_test'])
    test_obj2_dataset = TensorDataset(split_data['X_test'], split_data['y_obj2_test'])

    # create data loaders
    train_obj1_loader = DataLoader(train_obj1_dataset, batch_size=batch_size, shuffle=True)
    train_rel_loader = DataLoader(train_rel_dataset, batch_size=batch_size, shuffle=True)
    train_obj2_loader = DataLoader(train_obj2_dataset, batch_size=batch_size, shuffle=True)

    test_obj1_loader = DataLoader(test_obj1_dataset, batch_size=batch_size)
    test_rel_loader = DataLoader(test_rel_dataset, batch_size=batch_size)
    test_obj2_loader = DataLoader(test_obj2_dataset, batch_size=batch_size)

    return {
        'train': {
            'object_1': train_obj1_loader,
            'relation': train_rel_loader,
            'object_2': train_obj2_loader
            },
        'test': {
            'object_1': test_obj1_loader,
            'relation': test_rel_loader,
            'object_2': test_obj2_loader
            }
        }


In [4]:
#@title nonlinear probe

# class: nonlinear probe
class MLPProbe(nn.Module):
    def __init__(self, input_size: int, depth: int, width: int, output_size: int) -> None:
        super(MLPProbe, self).__init__()

        # define the layers
        layers = []
        for k in range(depth):
            if k == 0:
                layers.append(nn.Linear(input_size, width))
            else:
                layers.append(nn.Linear(width, width))
            layers.append(nn.ReLU())

        layers.append(nn.Linear(width, output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

In [5]:
#@title hyper-param tuning via optuna

def make_objective(input_dim: int, output_dim: int, train_loader: DataLoader, test_loader: DataLoader, epochs: int = 10):
    def objective(trial: optuna.Trial) -> float:

        # set device
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # hyper-params to tune
        depth = trial.suggest_int("depth", 2, 5)
        width = trial.suggest_int("width", 16, 256, log=True)
        learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
        # you can add dropout here later

        # define the model, loss, and optimizer
        model = MLPProbe(input_dim, depth, width, output_dim)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        # training loop
        model.train()
        total_loss = 0

        for epoch in range(epochs):
            epoch_loss = 0
            correct = 0
            total = 0

            for inputs, labels in train_loader:

                optimizer.zero_grad()

                # move data to device
                inputs, labels = inputs.to(device), labels.to(device)

                # forward pass
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # backward pass and optimize
                loss.backward()
                optimizer.step()

                # track metrics
                epoch_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            total_loss = epoch_loss / len(train_loader)
            accuracy = 100 * correct / total

        # log to W&B
        wandb.log({
            "trial": trial.number,
            "depth": depth,
            "width": width,
            "learning_rate": learning_rate,
            "loss": total_loss,
            "accuracy": accuracy})

        return total_loss

    return objective





In [6]:
#@title main

# load the data
VERBOSE = True
hdf5_path = "/content/drive/MyDrive/Llama-3.2-3B-Instruct_layer_embeddings_sample.h5"

data = load_data(hdf5_path, verbose=VERBOSE)

# parse sentences
pars_sentences = parse_sentences(data["sentences"], verbose=VERBOSE)

# encode triplets
labels, mappings = encode_triplets(pars_sentences["triplets"], verbose=VERBOSE)

loading data from HDF5 file...
Loaded 20000 sentences and their embeddings
Layer 8 embeddings shape: torch.Size([20000, 3072])
Layer 16 embeddings shape: torch.Size([20000, 3072])
Layer 24 embeddings shape: torch.Size([20000, 3072])
Data loading completed in 9.10 seconds
parsing sentences into triplets...
Successfully parsed 20000 triplets
Parsing completed in 0.04 seconds

Sample triplets:
0: ('table', 'over', 'chair')
1: ('table', 'on top of', 'chair')
2: ('table', 'higher than', 'chair')
3: ('table', 'elevated above', 'chair')
4: ('table', 'to', 'left of the chair')
Encoding triplets...
Found 15 unique objects as subject
Found 29 unique relations
Found 100 unique objects as object

Sample relations:
0: above
1: adjacent to
2: ahead of
3: at 45 degrees to
4: attached to
5: before
6: beside
7: close to
8: connected to
9: diagonally above-left
Encoding completed in 0.06 seconds


In [7]:
# define layer name and process layer_data
layer_name = "layer_8"
layer_data = data[layer_name]

if VERBOSE:
    print(f"\n{'='*20}")
    print(f"Processing {layer_name}")
    print(f"{'='*20}")

# prepare layer_data split
split_data = prepare_data_split(layer_data, pars_sentences["valid_indices"], labels, test_size=0.2)

# create data loaders
data_loaders = create_data_loaders(split_data, batch_size=1024)

# define component and input/output dimensions for training and hyper-param tuning
component = "object_1"
input_dim = layer_data.shape[1]
num_classes = len(mappings[component])

if VERBOSE:
    print(f"\nTraining probe for {component} classification ({num_classes} classes)...")


Processing layer_8

Training probe for object_1 classification (15 classes)...


In [8]:
# make the objective for optuna hyper-param tuning given component and layer

# define train and test loaders
train_loader = data_loaders["train"][component]
test_loader = data_loaders["test"][component]

# initialize weights & biases
wandb.login()
wandb.init(project="mlp-probe-optuna", entity="cmoyacal-purdue-university")

# create specialized objective function for optuna
objective = make_objective(input_dim, num_classes, train_loader, test_loader, epochs=5)


wandb: Currently logged in as: cmoyacal (cmoyacal-purdue-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [9]:
#@title run optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

# log best results to W&B
wandb.config.update(study.best_params)
wandb.finish()

print("Best hyperparameters:", study.best_params)

[I 2025-02-26 08:59:52,850] A new study created in memory with name: no-name-91df47ad-d891-4a95-9462-5dead7fb5a1f
[I 2025-02-26 09:00:01,971] Trial 0 finished with value: 0.0010572602550382726 and parameters: {'depth': 2, 'width': 144, 'learning_rate': 0.009106830531136447}. Best is trial 0 with value: 0.0010572602550382726.
[I 2025-02-26 09:00:03,898] Trial 1 finished with value: 2.2524874210357666 and parameters: {'depth': 3, 'width': 18, 'learning_rate': 0.0006593677919262046}. Best is trial 0 with value: 0.0010572602550382726.
[I 2025-02-26 09:00:05,714] Trial 2 finished with value: 2.2123638838529587 and parameters: {'depth': 5, 'width': 35, 'learning_rate': 0.0004557516309649238}. Best is trial 0 with value: 0.0010572602550382726.
[I 2025-02-26 09:00:09,704] Trial 3 finished with value: 0.00711760253761895 and parameters: {'depth': 3, 'width': 168, 'learning_rate': 0.0027979474929474895}. Best is trial 0 with value: 0.0010572602550382726.
[I 2025-02-26 09:00:11,279] Trial 4 finis

accuracy,█▃▂█▇▃▇▁█▇
depth,▁▃█▃▁██▆▁█
learning_rate,█▁▁▃▂▁▇▁▇▆
loss,▁▇▇▁▅▇▂█▁▂
trial,▁▂▃▃▄▅▆▆▇█
width,▅▁▂▆▁▇▁▂▂█
accuracy,89.7
depth,5
learning_rate,0.00695
loss,0.28307
trial,9


Best hyperparameters: {'depth': 2, 'width': 144, 'learning_rate': 0.009106830531136447}
